In [5]:
from flask import Flask, render_template
import sqlite3
import pandas as pd

app = Flask(__name__)

@app.route('/')
def index():
    conn = sqlite3.connect('nasa_data.db')
    neows_data = pd.read_sql('SELECT * FROM neows_data', conn).to_html(classes='table', index=False)
    meteorite_data = pd.read_sql('SELECT * FROM meteorite_landings', conn).to_html(classes='table', index=False)
    conn.close()
    return render_template('index.html', neows_data=neows_data, meteorite_data=meteorite_data)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.108:5000
Press CTRL+C to quit
127.0.0.1 - - [05/May/2024 15:40:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 15:40:01] "GET /static/miss_distance_vs_magnitude.png HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 15:40:01] "GET /static/potentially_hazardous_asteroids.png HTTP/1.1" 200 -
127.0.0.1 - - [05/May/2024 15:40:04] "GET /favicon.ico HTTP/1.1" 404 -
